In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.utils import check_random_state
import pickle
import math

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Data Science Project/processed_train_dataset.csv')

In [ ]:
train_df.head()

,Unnamed: 0,rating,helpful_vote,verified_purchase,sentiment,polarity,subjectivity,text_length,previous_rating
0,6384908,1.0,0,1,1,0.000000,0.000000,39,5.0
1,9567116,5.0,13,1,1,0.000000,0.000000,186,1.0
2,5749535,5.0,0,1,2,1.000000,0.300000,4,5.0
3,15464359,3.0,0,1,1,0.000000,0.333333,82,1.0
4,12386221,5.0,0,1,2,0.357143,0.728571,122,2.0


In [ ]:
train_df.shape

(5739360, 9)

In [ ]:
sentiment = np.array(list(train_df['sentiment']))

In [ ]:
train_df = train_df.drop(["sentiment", "Unnamed: 0"], axis=1)

In [ ]:
train_df.head()

,rating,helpful_vote,verified_purchase,polarity,subjectivity,text_length,previous_rating
0,1.0,0,1,0.000000,0.000000,39,5.0
1,5.0,13,1,0.000000,0.000000,186,1.0
2,5.0,0,1,1.000000,0.300000,4,5.0
3,3.0,0,1,0.000000,0.333333,82,1.0
4,5.0,0,1,0.357143,0.728571,122,2.0


In [ ]:
train_df.shape

(5739360, 7)

In [ ]:
train_data = train_df.iloc[:,:].values

In [ ]:
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)

In [ ]:
train_data_scaled.shape

(5739360, 7)

In [ ]:
def bernstein_inequality(n, epsilon, delta):
    return math.ceil((4 * n * np.log(1 / delta)) / (epsilon ** 2))

In [ ]:
k = 5
epsilon = 0.99
delta = 0.99

In [ ]:
n_s = bernstein_inequality(5739360, epsilon, delta)
print(n_s)

235415


In [ ]:
def compute_coreset(X, y, coreset_size, random_state=None):
    n_samples, _ = X.shape
    random_state = check_random_state(random_state)
    mean = np.mean(X, axis=0)
    distances = np.sum((X - mean) ** 2, axis=1)
    sensitivities = distances / np.sum(distances)
    # sensitivities += 1 / n_samples
    # sensitivities /= np.sum(sensitivities)
    sampled_indices = random_state.choice(np.arange(n_samples), size=coreset_size, replace=False, p=sensitivities)
    coreset_X = X[sampled_indices]
    coreset_Y = y[sampled_indices]
    coreset_weights = 1 / (coreset_size * sensitivities[sampled_indices])
    return coreset_X, coreset_Y, coreset_weights

In [ ]:
np.random.seed(42)
coreset_size = 235415
train_coreset, train_coreset_target, train_coreset_weights = compute_coreset(train_data_scaled, sentiment, coreset_size, random_state=42)

In [ ]:
train_coreset.shape

(235415, 7)

In [ ]:
train_coreset_target.shape

(235415,)

In [ ]:
train_coreset_weights.shape

(235415,)

In [ ]:
coreset_dict = {"coreset_matrix":train_coreset, 'labels':train_coreset_target, 'weights':train_coreset_weights}

In [ ]:
with open('/content/drive/MyDrive/Data Science Project/updated_coresets_train_dataset_230K.pkl', 'wb') as file:
    pickle.dump(coreset_dict, file)